# Load data


In [17]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Basic Perceptron model

Run the model:

$y = softmax\left(\mathbf{W}\vec{x} + \vec{b}\right)$

Train the model:

$H_{y'}(y) = -\sum_i y'_i \log(y_i)$

In [ ]:
# input node
x = tf.placeholder(tf.float32, [None, 784]) # shape = (instances, features)
# var containing weights
W = tf.Variable(tf.zeros([784, 10])) # (input features, classes)
# biases
b = tf.Variable(tf.zeros([10])) # 1 bias per class

y = tf.nn.softmax(tf.matmul(x, W) + b) # softmax(Wx + b)

with sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)):

# Using High-level API

In [20]:
feat_col = [tf.feature_column.numeric_column(key='X', shape=[784])]
clf = tf.contrib.learn.DNNClassifier(hidden_units=[256, 256], # two hidden layers
                                 feature_columns=feat_col,
                                 n_classes=10,
                                 model_dir='/tmp/mnist_clf')
# define training input data
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(mnist.train.images)},
    y=np.array(mnist.train.labels),
    num_epochs=None,
    shuffle=True)
# train the model
clf.fit(input_fn=train_input_fn, steps=2000)
# define testing input data
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(mnist.test.images)},
    y=np.array(mnist.test.labels),
    num_epochs=1,
    shuffle=False)

# nevermind... this is confusing

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_num_worker_replicas': 0, '_task_type': None, '_task_id': 0, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/mnist_clf', '_save_checkpoints_steps': None, '_tf_random_seed': None, '_environment': 'local', '_save_summary_steps': 100, '_session_config': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028E00043DA0>, '_save_checkpoints_secs': 600, '_master': ''}


ValueError: Feature X is not in features dictionary.

# Using Scikit-Learn

because tensorflow is ~~fucking hard~~ adds a layer of abstraction on top of everything

In [33]:
import sklearn.neural_network
from sklearn.datasets import load_digits
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

digits = load_digits()

n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1)) # shape is (n_samples, n_features)
cutoff = int(n_samples * 0.6) # using 60% of data for training
X_train = data[: cutoff] 
y_train = digits.target[: cutoff]
# MLPs are sensitive to feature scaling - need to scale inputs in training set
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)

X_test = data[cutoff :]
y_test = digits.target[cutoff :]
# also need to scale test data (scaler should only be fitted to training data!)
scaler.transform(X_test)

clf = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(256,256,256), # 3 hidden layers
                    activation='relu',
                    solver='adam',
                    batch_size='auto',
                    max_iter=200,
                    shuffle=True,
                    epsilon=1e-8)  # hyperparam for adam solver; default value=1e-8
clf.fit(X_train, y_train) # training in one line :-)
y_pred = clf.predict(X_test)
conf_mx = confusion_matrix(y_test, y_pred)
score = clf.score(X_test, y_test)

print(conf_mx)
print(score)


[[69  0  0  0  1  0  1  0  0  0]
 [ 0 66  1  2  0  0  0  0  1  3]
 [ 1  0 70  0  0  0  0  0  0  0]
 [ 0  0  1 65  0  2  0  1  5  0]
 [ 0  0  0  0 70  0  1  0  0  3]
 [ 0  0  0  0  0 70  1  0  0  0]
 [ 0  1  0  0  0  0 73  0  0  0]
 [ 0  0  0  0  0  0  0 70  2  0]
 [ 0  3  1  1  1  2  0  0 60  0]
 [ 0  0  0  1  0  2  0  0  0 68]]
0.947148817803
